In [172]:
import requests
import json
import math


def get_content_betano(game_url):
    r = requests.get(game_url)
    content = r.text
    return content


def get_bet_event(game_url):
    content = get_content_betano(game_url)
    token_mark = '<script>window["initial_state"]='
    inx_start = content.find(token_mark) + len(token_mark)
    inx_end = content.find("</script>", inx_start)
    json_content = content[inx_start:inx_end]
    dic_data = json.loads(json_content)
    bet_event = dic_data["data"]["event"]
    return bet_event


def get_players(event):
    match = event["name"]
    separator = " - "
    player1, player2 = str.split(match, separator)
    return player1, player2


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)
    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def find_by_name(value, lst):
    return list(filter(lambda x: x["name"] == value, lst))


def get_bet_target(token_class, bet_player, markets):
    winner_markets = find_by_name(token_class, markets)
    if len(winner_markets) > 0:
        winner_market = winner_markets[0]
    else:
        raise Exception(f"ERROR: {token_class} not found")

    tracked_player_list = find_by_name(bet_player, winner_market["selections"])
    if len(tracked_player_list) > 0:
        tracked_player_obj = tracked_player_list[0]
    else:
        raise Exception(f"ERROR: {bet_player} not found")
    return tracked_player_obj


def has_triggered_bet(tracked_player_obj, bet_value_expected):
    if tracked_player_obj["price"] >= bet_value_expected:
        msg = f'$ {tracked_player_obj["name"]} alcancou {bet_value_expected}: Atual {tracked_player_obj["price"]}'
        return msg


def is_second_half(event):
    try:
        period = event["liveData"]["periodDescription"]
        if period != "P1" and period != "P2":
            return True
    except:
        return


def can_bet_for_total_points_on_2nd_half(event):
    if is_second_half(event):
        first = event["liveData"]["results"]["firstPeriodScore"]
        second = event["liveData"]["results"]["secondPeriodScore"]
        firstHome, firstAway = first["home"], first["away"]
        secondHome, secondAway = second["home"], second["away"]
        totalFirstHalf = (
            int(firstHome) + int(firstAway) + int(secondHome) + int(secondAway)
        )
        markets = event["markets"]
        total_points = find_by_name("Total de Pontos", markets)
        less_than_points_obj = total_points[0]["selections"][1]
        handicap = less_than_points_obj["handicap"]
        projectedPoints = totalFirstHalf * 2
        can_bet = "NO" if projectedPoints - 3 >= handicap else "YES"
        # print(f"IT'S SECOND HALF - TOTAL POINTS:  -> can bet? {can_bet}")
        if can_bet == "YES":
            return "$ JOGAR EM TOTAL DE PONTOS PELO 2o Tempo: {totalFirstHalf}*2={projectedPoints} - hand: {handicap}"


from nba_api.stats.endpoints import *
import pandas


class TeamStats:
    def __init__(self, team_name, pontos):
        self.name = team_name
        self.media = pontos.mean()
        self.media_ultima = pontos[:10].mean()
        self.hot = pontos.sort_values(ascending=False)[:10].mean()
        self.cold = pontos.sort_values()[:10].mean()

    def pnts_esperados(self):
        res = (self.hot + self.media_ultima) / 2
        return math.floor(res)

    def pnts_esperados_baixo(self):
        res = (self.cold + self.media_ultima) / 2
        return math.floor(res)

    def __str__(self):
        return f"{self.name}:{self.pnts_esperados_baixo()}<-->{self.pnts_esperados()}"


def get_team_stats(team):
    teams_data = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
    team_id_res = teams_data[teams_data["TEAM_NAME"] == team]["TEAM_ID"]
    team_id = team_id_res.values[0]
    pontos = teamgamelogs.TeamGameLogs(
        season_nullable="2023-24", team_id_nullable=team_id
    ).get_data_frames()[0]["PTS"]
    return TeamStats(team, pontos)


def bet_pnts_by_player(player_stats, markets):
    try:
        player_tot_pnts = find_by_name(f"Total de Pontos {player_stats.name}", markets)
        player_tot_pnts = player_tot_pnts[0]["selections"]
        player_tot_pnts = player_tot_pnts + player_tot_pnts[1]["selections"]
    except:
        pass
    
    menor_pnts_do_jogo_provavel_player = player_stats.pnts_esperados_baixo()
    pnts_do_jogo_provavel_player1 = player_stats.pnts_esperados()
    for i in range(len(player_tot_pnts)):
        maior_que_curr = player_tot_pnts[i]
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel_player:
            preco = maior_que_curr["price"]
            print(
                f"$ JOGAR TOT_PNTS(ALT){player_stats.name} MAIOR QUE: {aposta_maior_que_curr}->{preco}/ESPERADO MAIOR: {menor_pnts_do_jogo_provavel_player}"
            )
            break

    for i in range(len(player_tot_pnts)):
        menor_que_curr = player_tot_pnts[i]
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > pnts_do_jogo_provavel_player1:
            preco = menor_que_curr["price"]
            print(
                f"$ JOGAR TOT_PNTS(ALT){player_stats.name} MENOR QUE: {aposta_menor_que_curr }->{preco}/ESPERADO MENOR: {pnts_do_jogo_provavel_player1}"
            )
            break
    return player_tot_pnts


def print_json(dic):
    print(json.dumps(dic))

In [173]:
import time
from plyer import notification
import pygame
import ctypes  # An included library with Python install.
import winsound

# Inicializa o mixer do pygame para sons
pygame.mixer.init()

def enviar_alerta(msg):
    # Exibe um popup
    # notification.notify(
    #     title='Alerta',
    #     message=msg,
    #     app_name='Temporizador'
    # )
    
    Mbox.BoxOkCancelAsync("ALERTA BET", msg)

    # Toca um som de alerta
    pygame.mixer.music.load('alarm.mp3')
    pygame.mixer.music.play()


class Mbox:
    OK = 1
    CANCELADO = 2

    def BoxOkCancel(title, text):
        winsound.MessageBeep(winsound.MB_OK)
        ctypes.windll.user32.MessageBoxW(0, text, title, 1)

    def Alerta(title, text=""):
        winsound.MessageBeep(winsound.MB_OK)
        funcao = lambda: ctypes.windll.user32.MessageBoxW(0, text, title, 0x1000)
        Mbox.executar_alerta_paralelo(funcao)

    def executar_alerta_paralelo(funcao_alerta, *args):
        thread_exec = ThreadSimples(funcao_alerta, *args)
        thread_exec.start()

    def BoxOkCancelAsync(title, text):
        Mbox.executar_alerta_paralelo(Mbox.BoxOkCancel, title, text)


In [174]:
import threading
import time
TEMPO_ALARME   =  30 # SEGUNDOS
class ThreadCustom(threading.Thread):
    def __init__(self, tempo, executar_funcao):
        super(ThreadCustom, self).__init__()
        self.parar = threading.Event()
        self.tempo = tempo
        self.executar = executar_funcao

    def run(self):
        while not self.parar.is_set():
            self.executar()
            time.sleep(self.tempo)

    def parar_execucao(self):
        print("Parando a execução...")
        self.parar.set()

def parar_thread(thread):
    thread.parar_execucao()
    thread.join()  # Aguarde até que a thread termine completamente

class ThreadSimples(threading.Thread):
    def __init__(self, funcao, *args, **kwargs):
        # If you are going to have your own constructor, make sure you call the parent constructor too!
        #        Thread.__init__(self)
        super().__init__()
        self.funcao = funcao
        self.kwargs = kwargs
        self.args = args

    def run(self):
        self.funcao(*self.args, **self.kwargs)

In [177]:
betano_url_games = [
    "https://br.betano.com/odds/phoenix-suns-sacramento-kings/44023448/?bt=8",
    "https://br.betano.com/live/philadelphia-76ers-denver-nuggets/44023440/",
]


def main():
    for url in betano_url_games:
        print(">>>>")
        process_game(url)


def process_game(game_url):
    try:
        event = get_bet_event(game_url)
    except:
        print(f"ERROR ON: {game_url}")
        return
    player1, player2 = get_players(event)
    p1_stats = get_team_stats(player1)
    p2_stats = get_team_stats(player2)
    pnts_do_jogo_provavel = p1_stats.pnts_esperados() + p2_stats.pnts_esperados()
    menor_pnts_do_jogo_provavel = (
        p1_stats.pnts_esperados_baixo() + p2_stats.pnts_esperados_baixo()
    )
    tot_pnts_str = f"TOT_PNTS: {math.floor(menor_pnts_do_jogo_provavel)}<-->{math.floor(pnts_do_jogo_provavel)}"
    print(f"> {player1} X {player2}: {tot_pnts_str}: {game_url}")
    tracked_player = player2
    bet_value_expected = 3
    markets = event["markets"]
    token_winner = "Vencedor"
    tracked_player_obj = get_bet_target(token_winner, tracked_player, markets)
    # print(f'{token_winner}:{tracked_player_obj["name"]}:{tracked_player_obj["price"]}')
    result = has_triggered_bet(tracked_player_obj, bet_value_expected)
    if result:
        enviar_alerta(result)

    bet_on_2nd_half = can_bet_for_total_points_on_2nd_half(event)
    if bet_on_2nd_half:
        enviar_alerta(bet_on_2nd_half)

    total_de_pontos = find_by_name("Total de Pontos", markets)
    menos_de_total_de_pontos = total_de_pontos[0]["selections"][1]
    aposta_tot_pnts = menos_de_total_de_pontos["handicap"]

    if aposta_tot_pnts > pnts_do_jogo_provavel:
        preco = menos_de_total_de_pontos["price"]
        print(f"$ JOGAR TOT_PNTS APOSTA: {aposta_tot_pnts}->{preco}")
    try:
        total_de_pontos_alternativas = total_de_pontos[1]
    except:
        total_de_pontos_alternativas = find_by_name(
            "Total de Pontos (alternativas)", markets
        )[0]
    rows = total_de_pontos_alternativas["tableLayout"]["rows"]

    preco_maior_que = lambda x: x["groupSelections"][0]["selections"][0]
    for i in range(len(rows)):
        maior_que_curr = preco_maior_que(rows[i])
        aposta_maior_que_curr = maior_que_curr["handicap"]
        if aposta_maior_que_curr < menor_pnts_do_jogo_provavel:
            preco = maior_que_curr["price"]
            print(f"$ JOGAR TOT_PNTS(ALT) MAIOR QUE: {aposta_maior_que_curr}->{preco}/ESPERADO MAIOR: {pnts_do_jogo_provavel}")
            break

    preco_menor_que = lambda x: x["groupSelections"][0]["selections"][1]
    for i in range(len(rows)):
        menor_que_curr = preco_menor_que(rows[i])
        aposta_menor_que_curr = menor_que_curr["handicap"]
        if aposta_menor_que_curr > pnts_do_jogo_provavel:
            preco = menor_que_curr["price"]
            print(f"$ JOGAR TOT_PNTS(ALT) MENOR QUE: {aposta_menor_que_curr }->{preco}/ESPERADO MENOR: {pnts_do_jogo_provavel}")
            break

    bet_pnts_by_player( p1_stats, markets)
    bet_pnts_by_player( p2_stats, markets)

main()

>>>>
> Phoenix Suns X Sacramento Kings: TOT_PNTS: 220<-->251: https://br.betano.com/odds/phoenix-suns-sacramento-kings/44023448/?bt=8
>>>>


KeyError: 'event'

In [ ]:

thread = ThreadCustom(TEMPO_ALARME,main)
thread.start()

In [ ]:
# parar_thread(thread)

Vencedor:Sacramento Kings:1.26
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 249.5 -> can bet? NO
Vencedor:Sacramento Kings:1.31
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.37
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.2
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 251.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 249.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.11
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - hand: 250.5 -> can bet? NO
Vencedor:Sacramento Kings:1.1
IT'S SECOND HALF - TOTAL POINTS: 133*2=266 - h

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\3542817686.py", line 13, in run
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\2787219941.py", line 15, in main
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11216\3992361451.py", line 84, in can_bet_for_total_points_on_2nd_half
IndexError: list index out of range


Vencedor:Sacramento Kings:1.007
